In [1]:
import os
import magic
import numpy as np
from secml.array import CArray

from secml_malware.attack.blackbox.c_wrapper_phi import CEmberWrapperPhi
from secml_malware.attack.blackbox.ga.c_base_genetic_engine import CGeneticAlgorithm
from secml_malware.models.c_classifier_ember import CClassifierEmber
from secml_malware.attack.blackbox.c_gamma_sections_evasion import CGammaSectionsEvasionProblem

In [2]:
net = CClassifierEmber(tree_path="../../modules/sota/ember/parameters/ember_model.txt")
net = CEmberWrapperPhi(net)

In [11]:
samples_goodware = "../samples_goodware/"
samples_testset = "../samples_testset/"

section_population, what_from_who = CGammaSectionsEvasionProblem.create_section_population_from_folder(samples_goodware, how_many=35, sections_to_extract=['.rdata', '.data', '.text'])

attack = CGammaSectionsEvasionProblem(section_population, net, population_size=30, penalty_regularizer=1e-6, iterations=15, threshold=0)

In [12]:
X = []
y = []
file_names = []

testset_samples = os.listdir(samples_testset)[0:2]
l = len(testset_samples)
for i, f in enumerate(testset_samples):
    link = os.path.join(samples_testset, f)
    path = magic.from_file(link).replace("symbolic link to `", "").strip("'")

    mmagic = magic.from_file(path)
    
    if "PE32" in mmagic:
        with open(path, "rb") as fhandle:
            code = fhandle.read()
    
        x = CArray(np.frombuffer(code, dtype=np.uint8)).atleast_2d()
        #x_features = net.extract_features(x)
        _, confidence = net.predict(x, True)
        y = confidence[1][0].item()
        print(path, y)

        if y > 0.5:
            print("> Initiating an attack")
            engine = CGeneticAlgorithm(attack)
            y_pred, adv_score, adv_ds, f_obj = engine.run(x, CArray(y))
            print(f_obj)

/data/quo.vadis/data/pe.dataset/PeX86Exe/clean/f916dfd05f97fb52c71c5f3c0adb639548b451c0d91db1618394e20df8680aa1 1.108668986490172e-07
/data/quo.vadis/data/pe.dataset/PeX86Exe/backdoor/d536c9da0ff9b846a9bbd91f54535f078496ccda6db7f081c4343838fac9c6fd 0.9999999645602727
> Initiating an attack
0.9999983284751466
